# Prediction model

After we perform feature engieering, now we will make preidction model and evaluartion.

In [1]:
import psycopg2
import pandas as pd
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
import re
import numpy as np

In [2]:
sql_nc = '''
select cs.cwhen as breaker
    , cm.cwhen as child
    , cs.message as breaker
    , cm.message as child
    , Substring(cs.csha,0,8) as broken
    , Substring(parent.prev,0,8) as parent 
    , Substring(child.curr,0,8) as child
    , cs.email as breaker
    , cm.email as child
from
impact_pairs as parent
, impact_pairs as child
, commits as cs
, commits as cm
,(
    (select prev as csha from impact_pairs) 
    union 
    (select curr as csha from impact_pairs)
) as ip
,(
    select cs.application as app
    , min(cwhen) as minwhen
    , max(cwhen) as maxwhen
    from 
    (
        (select prev as csha from impact_pairs) 
        union 
        (select curr as csha from impact_pairs)
    )
    as ip
    ,commits as cs
    ,(
        (select application,csha from findbugs_summary_uni) 
        union 
        (select application,csha from pmd_uni) 
        union 
        (select application,csha from sonarqube_system_uni) 
    )
    as anal
    where cs.csha=anal.csha
    and cs.csha=ip.csha
    and cs.application=anal.application
    and cs.application like %s
    group by cs.application
) as recovered
where
    cs.application = recovered.app
    and cs.cwhen <= recovered.maxwhen
    and cs.cwhen >= recovered.minwhen
    and cs.csha=ip.csha
    and parent.curr=cs.csha
    and child.prev=cs.csha
    and cm.csha=child.curr
    and ip.csha not in
    (
        select cs.csha
        from 
        (
            (select prev as csha from impact_pairs) 
            union 
            (select curr as csha from impact_pairs)
        )
        as ip
        ,commits as cs
        ,(
            (select application,csha from findbugs_summary_uni) 
            union 
            (select application,csha from pmd_uni) 
            union 
            (select application,csha from sonarqube_system_uni) 
        )
        as anal
        where cs.csha=anal.csha
        and cs.csha=ip.csha
        and cs.application=anal.application
    )
order by cs.application asc ,  cs.cwhen desc
'''

sql_c = '''
select cs.cwhen as breaker
    , cm.cwhen as child
    , cs.message as breaker
    , cm.message as child
    , Substring(cs.csha,0,8) as broken
    , Substring(parent.prev,0,8) as parent 
    , Substring(child.curr,0,8) as child
    , cs.email as breaker
    , cm.email as child
from
impact_pairs as parent
, impact_pairs as child
, commits as cs
, commits as cm
,(
    (select prev as csha from impact_pairs) 
    union 
    (select curr as csha from impact_pairs)
) as ip
,(
    select cs.application as app
    , min(cwhen) as minwhen
    , max(cwhen) as maxwhen
    from 
    (
        (select prev as csha from impact_pairs) 
        union 
        (select curr as csha from impact_pairs)
    )
    as ip
    ,commits as cs
    ,(
        (select application,csha from findbugs_summary_uni) 
        union 
        (select application,csha from pmd_uni) 
        union 
        (select application,csha from sonarqube_system_uni) 
    )
    as anal
    where cs.csha=anal.csha
    and cs.csha=ip.csha
    and cs.application=anal.application
    and cs.application like %s
    group by cs.application
) as recovered
where
    cs.application = recovered.app
    and cs.cwhen <= recovered.maxwhen
    and cs.cwhen >= recovered.minwhen
    and cs.csha=ip.csha
    and parent.curr=cs.csha
    and child.prev=cs.csha
    and cm.csha=child.curr
    and ip.csha in
    (
        select cs.csha
        from 
        (
            (select prev as csha from impact_pairs) 
            union 
            (select curr as csha from impact_pairs)
        )
        as ip
        ,commits as cs
        ,(
            (select application,csha from findbugs_summary_uni) 
            union 
            (select application,csha from pmd_uni) 
            union 
            (select application,csha from sonarqube_system_uni) 
        )
        as anal
        where cs.csha=anal.csha
        and cs.csha=ip.csha
        and cs.application=anal.application
    )
order by cs.application asc ,  cs.cwhen desc
'''

In [3]:
try:
    conn =  psycopg2.connect("dbname='icsme2018_march30' user='postgres' password = 'admin' host='localhost'")
except:
    print ("I am unable to connect to the database")  

cur = conn.cursor()

cur.execute(sql_nc, ('apache-%',))
pairs_nc = cur.fetchall()
cur.execute(sql_c, ('apache-%',))
pairs_c = cur.fetchall()

Features<br>
1) Message Similarity <br>
2) issue number (all of)<br>
3) time difference less than 1 hr?<br>
4) time messed up?<br>
5) same developer?<br>
6) Key word in commit message<br>


In [4]:
COMP = 0
NON_COMP = 1

df_nc = pd.DataFrame(pairs_nc, columns = ['break_time', 'child_time', 'break_message', 'child_message', 'break_csha', 'parrent_csha', 'child_csha', 'break_email', 'child_email'])
df_nc['compilable'] = NON_COMP
df_c = pd.DataFrame(pairs_c, columns = ['break_time', 'child_time', 'break_message', 'child_message', 'break_csha', 'parrent_csha', 'child_csha', 'break_email', 'child_email'])
df_c['compilable'] = COMP

df = pd.concat([df_nc, df_c])
df.head()

,break_time,child_time,break_message,child_message,break_csha,parrent_csha,child_csha,break_email,child_email,compilable
0,2016-03-03 07:07:10,2016-03-07 21:23:34,"[CALCITE-1124] Add TIMESTAMPADD, TIMESTAMPDIFF...","Further to [CALCITE-1124], add implementation ...",0b9ea98,cf5d07b,4ac82a3,arina.yelchiyeva@gmail.com,jhyde@apache.org,1
1,2015-12-07 17:47:31,2015-12-07 19:06:26,Move code from JdbcImplementor and JdbcRules t...,Remove redundant code from RelToSqlConverter\n...,980d9f8,eedd3dc,47e0e7c,jhyde@apache.org,jhyde@apache.org,1
2,2015-11-20 16:03:21,2015-12-05 14:36:02,[CALCITE-968] Stream-to-relation and stream-to...,Fix up [CALCITE-968]\n,e9d5060,8281668,937fc46,milinda.pathirage@gmail.com,jhyde@apache.org,1
3,2015-10-06 13:55:24,2015-10-06 14:09:59,"[CALCITE-785] Add ""Piglet"", a subset of Pig La...",Fix Piglet DUMP applied to multisets and struc...,5cee2a1,82ac7b2,26f303e,jhyde@apache.org,jhyde@apache.org,1
4,2015-07-09 11:46:58,2015-07-09 15:36:03,[CALCITE-783] Infer collation of Project using...,Complete [CALCITE-783] by fixing some planner ...,c711fed,f7ec3e8,9177063,milinda.pathirage@gmail.com,jhyde@apache.org,1


Feature 1 - Message Similarity

In [5]:
N_GRAMS = 2
THRESHOLD = 0.8

# def message_sim(row):
#     distance = 1 - jaccard_distance(set(ngrams(row['break_message'],N_GRAMS)), set(ngrams(row['child_message'],N_GRAMS)))
#     if distance > THRESHOLD:
#         return -1
#     else:
#         return 1




def message_sim(row):
    distance = 1 - jaccard_distance(set(ngrams(row['break_message'],N_GRAMS)), set(ngrams(row['child_message'],N_GRAMS)))
    return distance


df['feature_1'] = df.apply(message_sim, axis = 1)

Feature 2 - Issue Number

In [6]:
def issue_number(row):
    issue_number_break = re.findall('[A-Za-z]+[- ]\d+', row['break_message'])
    issue_number_child = re.findall('[A-Za-z]+[- ]\d+', row['child_message'])

    if len(issue_number_break) == 0 or len(issue_number_child) == 0:
        return -1
    elif issue_number_break == issue_number_child:
        return 1
    else:
        return -1

df['feature_2'] = df.apply(issue_number, axis = 1)

Feature 3 - Time Difference > 1 hour

In [7]:
HOUR = 3600
DAY = HOUR * 24

#df['feature_3'] = (df['child_time'] - df['break_time']).dt.total_seconds() > HOUR

df['feature_3'] = df.apply(lambda x: 1 if (x['child_time'] - x['break_time']).total_seconds() > HOUR else -1, axis =1)


Feature 4 - whether time messed up?

In [8]:
#df['feature_4'] = (df['child_time'] - df['break_time']).dt.total_seconds() > 0

df['feature_4'] = df.apply(lambda x: 1 if (x['child_time'] - x['break_time']).total_seconds() > 0 else -1, axis =1)

Feature 5 - whether same developer?

In [ ]:
df['feature_5'] = df['break_email'] == df['child_email']

Feature 6 - whether contain key words? <br>
['compile' 'compilation' '4280' 'sign' 'jakarta' 'missed' 'oops' 'missing' 'breakage' 'mgmt' 'adapted' 'wss4j' 'hamaconfiguration' 'branches' 'bad' 'service' 'conn' 'files' 'abstractserversession' 'month']

In [ ]:
# kw = ['missed' , 'oops']
# def contain_kw(row):
#     issue_number_break = re.findall('[A-Za-z]+[- ]\d+', row['break_message'])
#     if issue_number_break: return 1
#     else: return -1
    

def contain_kw(row):
    if  'missed' in  row['break_message'].lower(): 
        return 1
    else: return -1



def contain_fix(row):
    if 'javadoc' in row['break_message'].lower() : return 1
    else: return -1
    
    
# def contain_kw(row):
#     issue_number_break = re.findall('[A-Za-z]+[- ]\d+', row['break_message'])
#     if issue_number_break: return 1
#     else: return -1
    

df['feature_6'] = df.apply(contain_kw, axis = 1)
df['feature_8'] = df.apply(contain_fix, axis = 1)
df

Feature 7 - Log Time Difference 

In [ ]:
from sklearn.preprocessing import normalize

def log_(row):
    hour = (row['child_time'] - row['break_time']).total_seconds()/3600.0
    if hour >= 1.0/12:
        return np.log(hour)
    else: return np.log(1.0/12)

df['feature_7'] = df.apply(log_, axis =1)



In [ ]:
df.head(5)

Train the Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,confusion_matrix

In [ ]:
X = df[['feature_1', 'feature_2', 'feature_3', 'feature_4','feature_6','feature_7','feature_8']].iloc[0:1000]
y = df[['compilable']].iloc[0:1000]
X_train, X_test, y_train, y_test = train_test_split(X,y['compilable'], random_state = 0)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pre_proba = model.predict_proba(X_test)
y_pre = model.predict(X_test)
print('AUC: ', roc_auc_score(y_test, y_pre_proba[:,1]))
print('F1: ', f1_score(y_test, y_pre))
print(confusion_matrix(y_test, y_pre))

In [ ]:
model.coef_[0]